# DATA PROCESSING
* Cleaning & preprocessing data.
* Performing EDA

## Initialization

In [ ]:
# imports
import sqlalchemy as sa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from phpserialize import unserialize
from cleantext import clean
import hazm
import re
from matplotlib import style
import plotly.graph_objects as go
from cleaning_utils import clear_stop_char, replace_arabic_char

style.use("ggplot")

In [5]:
# Database connection
username = "root"
password = "rootpass"
host = "127.0.0.1"
port = 8081
db = "comment"
ch = "utf8"
db_engine = sa.create_engine(
    f"mysql://{username}:{password}@{host}:{port}/{db}?charset={ch}"
)
conn = db_engine.connect()

## Data Fetching

In [6]:
query = "select * from comments"
comment_df = pd.read_sql_query(query, conn)
print(comment_df.shape)
comment_df.head(3)

(1571618, 15)


,_id,id,lang,parent_id,options,model,category_id,master_id,object_master_id,creation,modification,user_id,published_by,unpublished_by,comment_data
0,1,813597,b'fa',NaN,3,8,145,2211391,2210271,2022-05-04 14:11:57,2022-05-04 14:24:03,NaN,5.0,NaN,"b'a:4:{s:5:""email"";N;s:7:""message"";s:172:"" \xd..."
1,2,813598,b'fa',NaN,3,8,145,2211392,2211048,2022-05-04 14:12:07,2022-05-04 14:24:01,NaN,5.0,NaN,"b'a:4:{s:5:""email"";N;s:7:""message"";s:37:""\xd8\..."
2,3,813599,b'fa',NaN,2,8,59,2211393,2210918,2022-05-04 14:13:12,2022-05-04 14:23:57,NaN,5.0,5.0,"b'a:4:{s:5:""email"";N;s:7:""message"";s:138:"" \xd..."


## Cleaning

### Unserializing the comments

In [7]:
def unserialize_func(blob):
    unserialized_blob = unserialize(blob)
    comment = unserialized_blob[b"message"].decode("utf-8")
    return comment

In [8]:
failed_counter = 0
for idx, row in comment_df.iterrows():
    try:
        comment_blob = row["comment_data"]
        comment = unserialize_func(comment_blob)
        comment_df.loc[idx,"comment_data"] = comment
    except Exception:
        failed_counter += 1    
print(f"Failed unserializing {failed_counter} comments")
comment_df = comment_df.drop_duplicates()
comment_df.head(3)

Failed unserializing 50 comments


,_id,id,lang,parent_id,options,model,category_id,master_id,object_master_id,creation,modification,user_id,published_by,unpublished_by,comment_data
0,1,813597,b'fa',NaN,3,8,145,2211391,2210271,2022-05-04 14:11:57,2022-05-04 14:24:03,NaN,5.0,NaN,تا نباشد چوبتر فرمان نبرد گاو وخر این ضرب‌ال...
1,2,813598,b'fa',NaN,3,8,145,2211392,2211048,2022-05-04 14:12:07,2022-05-04 14:24:01,NaN,5.0,NaN,دولتی شش کلاسی همینه
2,3,813599,b'fa',NaN,2,8,59,2211393,2210918,2022-05-04 14:13:12,2022-05-04 14:23:57,NaN,5.0,5.0,اگر بی حجابی علت گرانی بود پس چرا در زمان حکو...


### Cleaning comments

In [9]:
# Defining cleaner functions
def cleaning(text):
    text = text.strip()
    
    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )
    
    # normalizing
    normalizer = hazm.Normalizer()
    text = normalizer.normalize(text)
    
    # removing wierd patterns
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c"
        u"\u2068"
        u"\u2067"
        u"#"
        u"?"
        u"؟"
        u"."
        u"…"
        "]+", flags=re.UNICODE)
    
    text = wierd_pattern.sub(r' ', text)

    # Removing stop characters
    text = clear_stop_char(text)

    # Replacing arabic characters
    text = replace_arabic_char(text)

    # Removing repeating characters
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    
    # removing extra spaces, hashtags
    text = re.sub("\s+", " ", text)
    
    return text


In [10]:
# Cleaning
comment_df["cleaned_comment"] = comment_df["comment_data"].apply(cleaning)

# removing unnecessary columns
comment_df = comment_df[["cleaned_comment","published_by","unpublished_by"]].reset_index(drop=True)

# Removing empty string and splace comments
comment_df = comment_df[comment_df["cleaned_comment"]!= ""]
comment_df = comment_df[comment_df["cleaned_comment"]!= " "]

# Dropping duplicate columns
before_drop_length = len(comment_df)
comment_df = comment_df.drop_duplicates()
print(f"{before_drop_length - len(comment_df)} duplicate records removed.")
comment_df.head(3)

144939 duplicate records removed.


,cleaned_comment,published_by,unpublished_by
0,تا نباشد چوبتر فرمان نبرد گاو وخر این ضرب المث...,5.0,NaN
1,دولتی شش کلاسی همینه,5.0,NaN
2,اگر بی حجابی علت گرانی بود پس چرا در زمان حکوم...,5.0,5.0


## Preprocessing

### Generating status of the comment

In [ ]:
# Comments status
# The likely statuses are "not-reviewd", "rejected", "published"
not_reviewed = len(comment_df[comment_df["published_by"].isnull() & comment_df["unpublished_by"].isnull()])
rejected = len(comment_df[comment_df["published_by"].notnull() & comment_df["unpublished_by"].notnull()])
published = len(comment_df[comment_df["published_by"].notnull() & comment_df["unpublished_by"].isnull()])

# Showing the dist
fig = plt.figure(figsize=(16, 10))
plt.title("Comments Status")
plt.bar(["not reviewed", "rejected", "published"],height=[not_reviewed, rejected, published],color = ["gray", "#DC143C", "cyan"])
plt.yticks()
plt.yticks()
plt.show()

In [ ]:
# Rejected ration to published and whole
print(f"Ratio of rejected comments to whole: {(rejected / len(comment_df)) * 100}")
print(f"Ratio of rejected comments to published comments: {(rejected / published) * 100}")


In [11]:
# Changing status
comment_df.loc[comment_df["published_by"].isnull() & comment_df["unpublished_by"].isnull(), "status"] = "not_reviewed"
comment_df.loc[comment_df["published_by"].notnull() & comment_df["unpublished_by"].notnull(), "status"] = "rejected"
comment_df.loc[comment_df["published_by"].notnull() & comment_df["unpublished_by"].isnull(), "status"] = "published"

# Removing not_reviewed comments
print(f"Total records before removing not reviewed comments: {len(comment_df)}")
comment_df = comment_df[comment_df["status"]!="not_reviewed"]
print(f"Total records after removing not reviewed comments: {len(comment_df)}")


In [14]:
# Removing unnecessary columns
comment_df = comment_df[["cleaned_comment", "status"]].reset_index(drop=True)
comment_df.head()

,cleaned_comment,status
0,تا نباشد چوبتر فرمان نبرد گاو وخر این ضرب المث...,published
1,دولتی شش کلاسی همینه,published
2,اگر بی حجابی علت گرانی بود پس چرا در زمان حکوم...,rejected
3,خاک تو سرتون کنند که فقط ادعا دارید نه سواد و ...,published
4,دولت با 200 میلیون کردن قیمت پراید و دو تا سه ...,published


In [ ]:
# Adding status code
comment_df["status_code"] = comment_df["status"].apply(lambda x: 1 if x == "published" else 0)
comment_df.head(3)


### Best word length interval

In [16]:
comment_df['comment_len_by_words'] = comment_df['cleaned_comment'].apply(lambda t: len(hazm.word_tokenize(t)))
comment_df.head()

,cleaned_comment,status,comment_len_by_words
0,تا نباشد چوبتر فرمان نبرد گاو وخر این ضرب المث...,published,20
1,دولتی شش کلاسی همینه,published,4
2,اگر بی حجابی علت گرانی بود پس چرا در زمان حکوم...,rejected,16
3,خاک تو سرتون کنند که فقط ادعا دارید نه سواد و ...,published,21
4,دولت با 200 میلیون کردن قیمت پراید و دو تا سه ...,published,37


In [ ]:
min_max_len = comment_df["comment_len_by_words"].min(), comment_df["comment_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

In [ ]:
def data_gl_than(data, less_than=100.0, greater_than=0.0, col='comment_len_by_words'):
    data_length = data[col].values

    data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

    data_glt_rate = (data_glt / len(data_length)) * 100

    print(f'Texts with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')
    return data_glt_rate

In [ ]:
data_gl_than(comment_df,100,3)

In [ ]:
fig = plt.figure(figsize=(16, 10))
coverage = [None]
for i in range(2, 301):
    coverage.append(data_gl_than(comment_df,i,1))

plt.plot(coverage)
plt.xticks(range(0, 300, 20))
plt.yticks(range(0, 105, 5))
plt.show()

In [ ]:
fig = plt.figure(figsize=(16, 10))
coverage_begin = [None]

for i in range(1, 11):
    coverage_begin.append(data_gl_than(comment_df,100,i))
    
plt.plot(coverage_begin, color='cyan')
plt.xticks(range(1, 11, 1))
plt.yticks(range(0, 105, 5))
plt.show()

* Comments that have word count between `3` and `100` cover `91%` of the data

In [17]:
minlim, maxlim = 3, 100
# remove comments with the length of fewer than three words and longer than 100 words
comment_df['comment_len_by_words'] = comment_df['comment_len_by_words'].apply(lambda len_t: len_t if minlim <= len_t <= maxlim else None)
comment_df = comment_df.dropna(subset=['comment_len_by_words'])
comment_df = comment_df.reset_index(drop=True)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=comment_df['comment_len_by_words']
))

fig.update_layout(
    title_text='Distribution of word counts within comments',
    xaxis_title_text='Word Count',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

### Handling unbalanced data

In [ ]:
fig = go.Figure()

groupby_label = comment_df.groupby('status')['status'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_label.index)),
    y=groupby_label.tolist(),
    text=groupby_label.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of status within comments [DATA]',
    xaxis_title_text='Status',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [ ]:
# Selecting random number of published records
rejected_data = comment_df[comment_df['status_code'] == 0]
published_data = comment_df[comment_df['status_code'] == 1]

published_data = published_data.sample(n=len(rejected_data)).reset_index(drop=True)

new_data = pd.concat([published_data, rejected_data])
new_data = new_data.sample(frac=1).reset_index(drop=True)
new_data.info()

In [ ]:
fig = go.Figure()

groupby_label = new_data.groupby('status')['status'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_label.index)),
    y=groupby_label.tolist(),
    text=groupby_label.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of label within comments [NEW DATA]',
    xaxis_title_text='Status',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [ ]:
new_data = new_data[["cleaned_comment","status_code"]]
new_data.rename(columns={"cleaned_comment": "comment", "status_code": "status"}, inplace=True)
new_data.to_excel("full_cleaned_comment.xlsx")
new_data.head()

In [19]:
new_data = comment_df[["cleaned_comment","status"]]
new_data.rename(columns={"cleaned_comment": "comment", "status": "status"}, inplace=True)
new_data.to_csv("all_full_cleaned_comment.csv")
new_data.head()

/tmp/ipykernel_22389/1046203210.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data.rename(columns={"cleaned_comment": "comment", "status": "status"}, inplace=True)


,comment,status
0,تا نباشد چوبتر فرمان نبرد گاو وخر این ضرب المث...,published
1,دولتی شش کلاسی همینه,published
2,اگر بی حجابی علت گرانی بود پس چرا در زمان حکوم...,rejected
3,خاک تو سرتون کنند که فقط ادعا دارید نه سواد و ...,published
4,دولت با 200 میلیون کردن قیمت پراید و دو تا سه ...,published


: 